In [1]:
import numpy as np
import scipy.io
import os
from string import Template


root_dir = '/Users/patrick/Desktop/emopic_fMRI/preproc/PREPROC'


# Conditions in .mat files
# Negs, Pos, Neu


# Contrasts of interest are:
# Neg_minus_Neu: 0, 1, -1, 0
# Pos_minus_Neu: 0, 0, -1, 1
 

def get_contrasts():
    contrasts = {}
    contrasts['names'] = ['Neg_minus_Neu','Pos_minus_Neu'] #conds=[x,neg,neu,pos]
    contrasts['vectors'] = list([
        np.array([0, 1, -1, 0], dtype=np.double),
        np.array([0, 0, -1, 1], dtype=np.double)
    ])
    return contrasts


def write_contrasts(filepath, ctrst_data):
    names = ctrst_data['names']
    vectors = ctrst_data['vectors']
    np_vectors = np.empty((len(names),), dtype=np.object)
    for i in range(len(names)):
        np_vectors[i] = vectors[i]

    mat = {}
    mat['tcon_name'] = np.array(names, dtype=np.object)
    mat['tcon_vec'] = np.array(np_vectors, dtype=np.object)
    try:
        scipy.io.savemat(filepath, mat)
    except:
        print('error saving file:' + filepath)
        return False

    return True

# Save contrasts data

ctrst_file = os.path.join(root_dir, 'contrasts.mat')
ctrst_data = get_contrasts()
print(ctrst_data)
write_contrasts(ctrst_file, ctrst_data)

{'names': ['Neg_minus_Neu', 'Pos_minus_Neu'], 'vectors': [array([ 0.,  1., -1.,  0.]), array([ 0.,  0., -1.,  1.])]}


True

In [2]:
SPM_TEMPLATE = """

spm_path = '${spm_path}'; 
fmri_path = '${fmri_file}'; 
cond_path = '${cond_file}'; 
cont_path = '${cont_file}'; 
rgr_path = '${rgr_file}'; 
tr = ${tr}; %2
scan_count = ${scan_count}; %150

% Initialize SPM
addpath(spm_path);
spm('Defaults', 'fMRI');
spm_jobman('initcfg');

% Open a graphical window for SPM with the tag Graphics
spm_figure('Create', 'Graphics', 'SPM12');

% Get volume list
[fmri_dir fmri_file fmri_ext] = fileparts(fmri_path);
fmri_scans = {};
for f = 1:150
    fmri_scans{f} = fullfile(fmri_dir, [fmri_file, fmri_ext, ',' num2str(f)]);
end
fmri_scans = cellstr(char(fmri_scans));
if length(fmri_scans) ~= scan_count
    error('incorrect count of fMRI scans')
end

% Job 1 : Specify First-Level
matlabbatch{1}.spm.stats.fmri_spec.dir = {fmri_dir};
matlabbatch{1}.spm.stats.fmri_spec.timing.units = 'secs';
matlabbatch{1}.spm.stats.fmri_spec.timing.RT = tr;
matlabbatch{1}.spm.stats.fmri_spec.timing.fmri_t = 16;
matlabbatch{1}.spm.stats.fmri_spec.timing.fmri_t0 = 1;
matlabbatch{1}.spm.stats.fmri_spec.sess.scans = fmri_scans;
matlabbatch{1}.spm.stats.fmri_spec.sess.cond = struct('name', {}, 'onset', {}, 'duration', {}, 'tmod', {}, 'pmod', {}, 'orth', {});
matlabbatch{1}.spm.stats.fmri_spec.sess.multi = {cond_path};
matlabbatch{1}.spm.stats.fmri_spec.sess.regress = struct('name', {}, 'val', {});
matlabbatch{1}.spm.stats.fmri_spec.sess.multi_reg = {rgr_path};
matlabbatch{1}.spm.stats.fmri_spec.sess.hpf = 128;
matlabbatch{1}.spm.stats.fmri_spec.fact = struct('name', {}, 'levels', {});
matlabbatch{1}.spm.stats.fmri_spec.bases.hrf.derivs = [0 0];
matlabbatch{1}.spm.stats.fmri_spec.volt = 1;
matlabbatch{1}.spm.stats.fmri_spec.global = 'None';
matlabbatch{1}.spm.stats.fmri_spec.mthresh = 0.8;
matlabbatch{1}.spm.stats.fmri_spec.mask = {''};
matlabbatch{1}.spm.stats.fmri_spec.cvi = 'AR(1)';

% Save the batch 
save(fullfile(fmri_dir, ['firstlevel_specify.mat']), 'matlabbatch');

% Run preprocessing batch job
spm_jobman('run', matlabbatch);

% Name .ps using sequential number and current script name
pdf_dir = fmri_dir;
dir_list = dir(fullfile(pdf_dir, '*.ps'));
ps_count = length(dir_list);
new_ps = fullfile(pdf_dir,[num2str(ps_count) '-firstlevel_specify.ps'])
spm_print(new_ps);


clear matlabbatch;

% loads tcon_name and tcon_vec
load(cont_path);

% Job 1 : Estimate
matlabbatch{1}.spm.stats.fmri_est.spmmat = {fullfile(fmri_dir, 'SPM.mat')};
matlabbatch{1}.spm.stats.fmri_est.method.Classical = 1;

% Job 2 : Contrasts
matlabbatch{2}.spm.stats.con.spmmat = {fullfile(fmri_dir, 'SPM.mat')};
for cur_tcon = 1:numel(tcon_name);                  
    matlabbatch{2}.spm.stats.con.consess{cur_tcon}.tcon.name = tcon_name{cur_tcon};
    matlabbatch{2}.spm.stats.con.consess{cur_tcon}.tcon.convec = tcon_vec{cur_tcon};
    matlabbatch{2}.spm.stats.con.consess{cur_tcon}.tcon.sessrep = 'none';
end        
matlabbatch{2}.spm.stats.con.delete = 0;

% Job 3 : Results
matlabbatch{3}.spm.stats.results.spmmat = {fullfile(fmri_dir, 'SPM.mat')};
matlabbatch{3}.spm.stats.results.conspec.titlestr = '';
matlabbatch{3}.spm.stats.results.conspec.contrasts = Inf;
matlabbatch{3}.spm.stats.results.conspec.threshdesc = 'none';
matlabbatch{3}.spm.stats.results.conspec.thresh = 0.005;
matlabbatch{3}.spm.stats.results.conspec.extent = 0;
matlabbatch{3}.spm.stats.results.conspec.mask = struct('contrasts', {}, 'thresh', {}, 'mtype', {});
matlabbatch{3}.spm.stats.results.units = 1;
matlabbatch{3}.spm.stats.results.print = true;

% Save the batch 
save(fullfile(fmri_dir, ['firstlevel_results.mat']), 'matlabbatch');

% Run preprocessing batch job
spm_jobman('run', matlabbatch);

% Save the PS file
dir_list = dir(fullfile(pwd,'spm*.ps'));
if (length(dir_list) == 0)
    fprintf('ERROR:failed to find .ps file')
else
    % Get name of current spm .ps
    cur_ps = fullfile(pwd, dir_list(1).name);

    % Rename current spm .ps using sequential number and current script name
    dir_list = dir(fullfile(pdf_dir, '*.ps'));
    ps_count = length(dir_list);
    new_ps = fullfile(pdf_dir,[num2str(ps_count) '-firstlevel_results.ps'])
    movefile(cur_ps, new_ps)
end

spm_clf;

% Combine PDFs
[status,msg] = system([ ...
    'gs -dBATCH -dNOPAUSE -sDEVICE=pdfwrite -sOutputFile=' ...
    pdf_dir '/report.pdf ' ...
    pdf_dir '/*.ps ']);
if status~=0
    warning('Could not cleanly create PDF file.');
    disp(msg);
end

disp('DONE');

EOF
"""


SESS_LIST = ['2201a', '2202a', '2203a', '2204a', '2205a', '2207a', '2208a', '2209a', '2210a', '2211a', '2212a', 
'2213a', '2214a', '2215a', '2216a', '2217a', '2218a', '2219a', '2220a', '2221a', '2222a', '2223a', 
'2224a', '2225a', '2226a', '2227a', '2228a', '2251a', '2252a', '2253a', '2254a', '2255a', '2256a', 
'2257a', '2258a', '2261a', '2262a', '2264a', '2265a', '2266a', '2267a', '2268a', '2269a', '2270a',
'2271a', '2274a', '2275a', '2276a', '2277a', '2278a', '2279a'];

for sess in sorted(SESS_LIST):
    print(sess)
    
    fmri_file = os.path.join(root_dir, sess, 'swau' + sess + '_EMOPIC.nii')
    cond_file = os.path.join('/Users/patrick/Desktop/emopic_fMRI/ENCODING1', sess + '_conditions.mat')
    rgr_file = os.path.join(root_dir, sess, 'art_regression_outliers_and_movement_au' + sess + '_EMOPIC.mat')
    cont_file = os.path.join(root_dir, 'contrasts.mat')
    tr = '2'
    scan_count = 150
    save_path = os.path.join(root_dir, sess)
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    
    cmd_file = os.path.join(root_dir, sess, 'cmd.txt')
        
    spm_code = Template(SPM_TEMPLATE).substitute({
        'fmri_file': fmri_file,
        'cond_file': cond_file,
        'rgr_file': rgr_file,
        'cont_file': cont_file,
        'spm_path': '/Users/patrick/Desktop/spm12',
        'tr': tr,
        'scan_count': scan_count})
        
    cmd = '/Applications/MATLAB_R2020a.app/bin/matlab -singleCompThread  -nodesktop -nosplash << EOF '+spm_code
        
    # Save it
    with open(cmd_file, 'w') as f:
        f.write(cmd)
        
    # Run it
    os.system(cmd)

print('All DONE!')

2201a
2202a
2203a
2204a
2205a
2207a
2208a
2209a
2210a
2211a
2212a
2213a
2214a
2215a
2216a
2217a
2218a
2219a
2220a
2221a
2222a
2223a
2224a
2225a
2226a
2227a
2228a
2251a
2252a
2253a
2254a
2255a
2256a
2257a
2258a
2261a
2262a
2264a
2265a
2266a
2267a
2268a
2269a
2270a
2271a
2274a
2275a
2276a
2277a
2278a
2279a
All DONE!


In [25]:
mkdir(/Users/patrick/Desktop/emopic_fMRI/preproc/PREPROC/2201a)

/bin/sh: -c: line 0: syntax error near unexpected token `/Users/patrick/Desktop/emopic_fMRI/preproc/PREPROC/2201a'
/bin/sh: -c: line 0: `mkdir (/Users/patrick/Desktop/emopic_fMRI/preproc/PREPROC/2201a)'


In [32]:
path = os.path.join(root_dir)
if os.path.exists(save_path):
    print(path + ' :exists')
    if os.path.isdir(save_path):
        print(save_path + ' : is a directory')

/Users/patrick/Desktop/emopic_fMRI/preproc/PREPROC :exists
/Users/patrick/Desktop/emopic_fMRI/preproc/PREPROC : is a directory


In [33]:
save_path = os.path.join(root_dir, sess)
print(save_path)

/Users/patrick/Desktop/emopic_fMRI/preproc/PREPROC/2201a


In [43]:
mfile_dir = '/Users/patrick/Desktop/emopic_fMRI/preproc/PREPROC/RunPreproc_Template.m';
addpath(fullfile(mfile_dir, 'ext'));

spm_path = '${spm_path}'; '/Users/patrick/Desktop/spm12/'
fmri_path = '${fmri_file}'; '/Users/patrick/Desktop/emopic_fMRI/preproc/PREPROC/'
cond_path = '${cond_file}'; '/Users/patrick/Desktop/emopic_fMRI/preproc/PREPROC/'
cont_path = '${cont_file}'; '/Users/patrick/Desktop/emopic_fMRI/preproc/PREPROC/'
rgr_path = '${rgr_file}'; '/Users/patrick/Desktop/emopic_fMRI/preproc/PREPROC/'
tr = '${tr}'; 2
scan_count = '${scan_count}'; 150

% Initialize SPM
addpath(spm_path);
spm('Defaults', 'fMRI');
spm_jobman('initcfg');

% Open a graphical window for SPM with the tag Graphics
spm_figure('Create', 'Graphics', 'SPM12');

% Load header and get params
fhdr = load_nii_hdr(fmri_path);
nfmri_scan = fhdr.dime.dim(5);

NameError: name 'addpath' is not defined